In [31]:
import pandas as pd
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches

from matplotlib import image as mpimg
from matplotlib.legend_handler import HandlerBase
from matplotlib.offsetbox import OffsetImage, DrawingArea

ALPHA_VALUE = 1.0
PARAM_LIMS = [-1, 1]
OBSERVABLES = ["X", "Y", "Z"]
PARENT_DIR = "../qfs_paper/figures/"

SMALL_FONTSIZE = 18
MEDIUM_FONTSIZE = SMALL_FONTSIZE + 2
LARGE_FONTSIZE = MEDIUM_FONTSIZE + 2

plt.rc("font", family="serif", serif="cm10")
plt.rc("text", usetex=True)


In [32]:
class ImageHandler(HandlerBase):
    def __init__(self, image, zoom=1.0, horizontal_pad=0, vertical_pad=0):
        self.image = image
        self.zoom = zoom
        self.horizontal_pad = horizontal_pad
        self.vertical_pad = vertical_pad
        super().__init__()

    def legend_artist(self, legend, orig_handle, fontsize, handlebox):
        # Create OffsetImage
        imagebox = OffsetImage(self.image, zoom=self.zoom)
        imagebox.image.axes = None  # Detach from axes

        # Wrap the OffsetImage in a DrawingArea with padding
        padded_area = DrawingArea(
            width=0,
            height=0,
            xdescent=0,
            ydescent=0,
        )
        padded_area.add_artist(imagebox)
        imagebox.set_offset((self.horizontal_pad, self.vertical_pad))

        handlebox.add_artist(padded_area)

        return padded_area

In [33]:
use_pdf = True
use_png = not use_pdf
if use_pdf:
    mpl.use("pdf")
file_extension = "pdf" if use_pdf else "png"

In [34]:
observable_indentity_parameters = {
    "X": [1, 0.0, 0.0],
    "Y": [0.0, 1, 0.0],
    "Z": [0.0, 0.0, 1],
}

In [35]:
data_frame1 = pd.read_csv(
    "../data_csv_files/data_for_increasing_noise_one_on_f.csv",
    index_col=0,
)

noise_strs = data_frame1.index.values
data_frame1 = data_frame1.to_numpy()

data_frame2 = pd.read_csv(
    "../data_csv_files/data_for_increasing_noise_coloured_noise_ns.csv",
    index_col=0,
).to_numpy()

In [36]:
fig = plt.figure(figsize=(20, 6))

fig.suptitle(
    r"$\tilde{O}$ Space With Increasing Noise Strength (C)",
    fontsize=LARGE_FONTSIZE,
)

cmap_red = cm.Reds
cmap_blue = cm.Blues
norm = mcolors.Normalize(vmin=0, vmax=len(data_frame1))

for observable_index, observable in enumerate(OBSERVABLES):
    ax = fig.add_subplot(1, 3, observable_index + 1, projection="3d")
    ax.clear()
    ax.set_title(r"$\tilde{{{}}}$".format(observable), fontsize=MEDIUM_FONTSIZE)

    ax.set_xlim3d(*PARAM_LIMS)
    ax.set_ylim3d(*PARAM_LIMS)
    ax.set_zlim3d(*PARAM_LIMS)
    ax.set_xlabel(r"$\alpha$", fontsize=SMALL_FONTSIZE)
    ax.set_ylabel(r"$\beta$", fontsize=SMALL_FONTSIZE)
    ax.set_zlabel(r"$\gamma$", fontsize=SMALL_FONTSIZE)
    ax.set_xticks([PARAM_LIMS[0], 0, PARAM_LIMS[1]])
    ax.set_yticks([PARAM_LIMS[0], 0, PARAM_LIMS[1]])
    ax.set_zticks([PARAM_LIMS[0], 0, PARAM_LIMS[1]])
    ax.tick_params(labelsize=16)

    base_idx = observable_index * 3

    for idx, profile in enumerate(data_frame1):
        if idx == 0:
            colour = cmap_red(0.2)
        else:
            colour = cmap_red(norm(idx + 1))
        ax.scatter(
            profile[base_idx],
            profile[base_idx + 1],
            profile[base_idx + 2],
            alpha=ALPHA_VALUE,
            s=100,
            color=colour,
        )

    for idx, profile in enumerate(data_frame2):
        if idx == 0:
            colour = cmap_blue(0.2)
        else:
            colour = cmap_blue(norm(idx + 1))
        ax.scatter(
            profile[base_idx],
            profile[base_idx + 1],
            profile[base_idx + 2],
            alpha=ALPHA_VALUE,
            s=100,
            color=colour,
        )

    ax.scatter(
        observable_indentity_parameters[observable][0],
        observable_indentity_parameters[observable][1],
        observable_indentity_parameters[observable][2],
        alpha=ALPHA_VALUE,
        s=100,
        color="green",
    )

red_grad = mpimg.imread("./red_grad.png")
blue_grad = mpimg.imread("./blue_grad.png")
red_image_handle = "Red Image"
blue_image_handle = "Blue Image"
green_patch = mpatches.Patch(color="green", label="Identity")

fig.legend(
    handles=[red_image_handle, blue_image_handle, green_patch],
    labels=[r"1/f", "Coloured", "Identity"],
    handler_map={
        red_image_handle: ImageHandler(
            red_grad, zoom=0.06, horizontal_pad=380, vertical_pad=20
        ),
        blue_image_handle: ImageHandler(
            blue_grad, zoom=0.04, horizontal_pad=495, vertical_pad=20
        ),
    },
    loc="lower center",
    ncol=3,
    bbox_to_anchor=(0.525, -0.125),
    title_fontsize=MEDIUM_FONTSIZE,
    prop={"size": MEDIUM_FONTSIZE},
)

fig.subplots_adjust(hspace=0.4)

if use_pdf:
    plt.savefig(f"{PARENT_DIR}vis_increasing_noise_str.pdf", bbox_inches="tight")
    plt.close()
else:
    plt.show()
    plt.close()